# 1. Importing Libraries and reading the data

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import numpy as np
import pandas as pd
import string
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
import nltk
from nltk.corpus import stopwords
from nltk.stem import SnowballStemmer
import seaborn as sns
from plotly import graph_objs as go
import matplotlib as plt

from wordcloud import WordCloud, STOPWORDS, ImageColorGenerator
from PIL import Image

In [ ]:
sms = pd.read_csv("/content/drive/MyDrive/spam_det/spam.csv",encoding = "latin")
sms.head()

,v1,v2,Unnamed: 2,Unnamed: 3,Unnamed: 4
0,ham,"Go until jurong point, crazy.. Available only ...",NaN,NaN,NaN
1,ham,Ok lar... Joking wif u oni...,NaN,NaN,NaN
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...,NaN,NaN,NaN
3,ham,U dun say so early hor... U c already then say...,NaN,NaN,NaN
4,ham,"Nah I don't think he goes to usf, he lives aro...",NaN,NaN,NaN


2. Data Cleaning

In [ ]:
sms = sms.rename(columns = {'v1':'Spam/Not_Spam','v2':'message'})

In [ ]:
sms.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5572 entries, 0 to 5571
Data columns (total 5 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   Spam/Not_Spam  5572 non-null   object
 1   message        5572 non-null   object
 2   Unnamed: 2     50 non-null     object
 3   Unnamed: 3     12 non-null     object
 4   Unnamed: 4     6 non-null      object
dtypes: object(5)
memory usage: 217.8+ KB


In [ ]:
sms.groupby('Spam/Not_Spam').describe()

message         \
                count unique   
Spam/Not_Spam                  
ham              4825   4516   
spam              747    653   

                                                                       \
                                                             top freq   
Spam/Not_Spam                                                           
ham                                       Sorry, I'll call later   30   
spam           Please call our customer service representativ...    4   

              Unnamed: 2         \
                   count unique   
Spam/Not_Spam                     
ham                   45     39   
spam                   5      4   

                                                                       \
                                                             top freq   
Spam/Not_Spam                                                           
ham             bt not his girlfrnd... G o o d n i g h t . . .@"    3   
spam                                                 PO Box 5249    2   

              Unnamed: 3                                    Unnamed: 4         \
                   count unique                    top freq      count unique   
Spam/Not_Spam                                                                   
ham                   10      9                     GE    2          6      5   
spam                   2      1   MK17 92H. 450Ppw 16"    2          0      0   

                             
                   top freq  
Spam/Not_Spam                
ham            GNT:-)"    2  
spam               NaN  NaN

In [ ]:
sms_copy = sms['message'].copy()

In [ ]:
def text_preprocess(text):
    text = text.translate(str.maketrans('', '', string.punctuation))
    text = [word for word in text.split() if word.lower() not in stopwords.words('english')]
    return " ".join(text)

In [ ]:
import nltk
nltk.download('stopwords')


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [ ]:
sms_copy = sms_copy.apply(text_preprocess)

In [ ]:
sms_copy

0       Go jurong point crazy Available bugis n great ...
1                                 Ok lar Joking wif u oni
2       Free entry 2 wkly comp win FA Cup final tkts 2...
3                     U dun say early hor U c already say
4             Nah dont think goes usf lives around though
                              ...                        
5567    2nd time tried 2 contact u U å£750 Pound prize...
5568                          Ì b going esplanade fr home
5569                          Pity mood Soany suggestions
5570    guy bitching acted like id interested buying s...
5571                                       Rofl true name
Name: message, Length: 5572, dtype: object

In [ ]:
vectorizer = TfidfVectorizer()

In [ ]:
sms_mat = vectorizer.fit_transform(sms_copy)
sms_mat

<5572x9376 sparse matrix of type '<class 'numpy.float64'>'
	with 47254 stored elements in Compressed Sparse Row format>

In [ ]:
message_train, message_test, spam_nospam_train, spam_nospam_test = train_test_split(sms_mat, sms['Spam/Not_Spam'], test_size=0.3, random_state=20)

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

Spam_model = LogisticRegression(solver='liblinear', penalty='l1')
Spam_model.fit(message_train, spam_nospam_train)
pred = Spam_model.predict(message_test)
accuracy_score(spam_nospam_test,pred)

0.9383971291866029